In [1]:
!pip install transformers[torch]

In [2]:
import os
import shutil

import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset
from transformers import (
    TrainerCallback, 
    EarlyStoppingCallback, 
    AutoTokenizer, 
    T5ForConditionalGeneration, 
    AutoModelForSeq2SeqLM, 
    DataCollatorForSeq2Seq, 
    Seq2SeqTrainingArguments, 
    Seq2SeqTrainer
)

np.random.seed(0)

2024-02-17 16:44:09.612581: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-17 16:44:09.673884: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-17 16:44:09.673978: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-17 16:44:09.675717: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-17 16:44:09.686345: I tensorflow/core/platform/cpu_feature_guar

In [3]:
user_questions = [
    "How many orders were placed in City A?",
    "What is the average price of products in the Electronics category?",
    "Show me the orders with a review score of 5.",
    "What is the total quantity of Toys sold?",
    "List all orders made by user ID 3.",
    "How many orders were placed in City A?",
    "What is the average price of products in the Electronics category?",
    "Show me the orders with a review score of 5.",
    "What is the total quantity of Toys sold?",
    "List all orders made by user ID 3.",
    "What is the total price of orders made by user ID 2?",
    "How many unique product categories are there?",
    "List top 3 orders with the highest review scores.",
    "What is the minimum price of products?",
    "How many orders were placed on 2024-02-13?",
    "How many orders were placed in City A?",
    "What is the average price of products in the Electronics category?",
    "Show me the orders with a review score of 5.",
    "What is the total quantity of Toys sold?",
    "List all orders made by user ID 3.",
    "What is the total price of orders made by user ID 2?",
    "How many unique product categories are there?",
    "List top 3 orders with the highest review scores.",
    "What is the minimum price of products?",
    "How many orders were placed on 2024-02-13?",
    "What is the maximum quantity of products sold in a single order?",
    "List orders with a quantity greater than 2.",
    "How many orders were made in February 2024?",
    "What is the average review score for orders with more than 1 product?",
    "Show me the orders made by user ID 4 in City A.",
    "List orders made on 2024-02-12 with a review score of 4 or higher.",
    "What is the total price of orders with a review score of 5?",
    "List orders with a price greater than $50 and a review score of 4 or higher.",
    "How many users made more than one order?",
    "What is the average price of products for each user?",
    "Show me the orders made by user ID 1 before 2024-02-11.",
    "What is the most expensive product sold?",
    "List orders with the highest and lowest prices.",
    "What is the total quantity of products sold in City B?",
    "List orders made by users from City C with a review score of 5.",
    "How many orders were placed between 2024-02-10 and 2024-02-12?",
    "What is the average price of products in each product category?",
    "Show me the orders made on weekends.",
    "List orders with a quantity less than the average quantity.",
    "How many orders were made by each user?",
    "What is the total revenue generated?",
    "List orders with a price higher than the average price.",
    "What is the total quantity of products sold for each product category?",
    "How many orders were placed in City A?",
    "What is the average price of products in the Electronics category?",
    
    "How many total interactions are observed?",
    "How many total interactions are observed in the year 2010?",
    "How many total observations are there?",
    "How many interactions happened in the last year?",
    "How many interactions happened in the last 5 years?",
    "How many interactions happened in the last three years?",
    "How many average interactions are observed in a year?",
    "What is the total transaction count?",
    "What is the transaction count?",
    "What is the transaction volume observed over the years?",
    "How many transactions happened in the last month?",
    "How many transactions took place on the 15th of last month?",
    "How many orders were placed on 20th Feb 2012?",
    "How many transactions were observed between 2nd Mar 2011 and 31st Mar 2012?",
    "List all the order details in the last week?",
    
    "What is our current customer volume?",
    "How many customers made transactions in the last 3 months?",
    "What is the customer base?",
    "How many customers belongs to city XYZ?",
    "How many customers are in each city?",
    "How many customers in London?",
    "What is the average sale for customer X?",
    "List top 5 customers with highest basket value in last week?",
    "which customer has bought maximum items yesterday from Eastbourne?",
    "what is the average basket value on a Friday?",
    "which year has the highest sales?",
    "which customer has done the highest transaction and has given the highest rating?",
    "In how many cities are we present?",
    "which city has the highest sale since May LY?",
    "what is the max no. of items bought by a customer on a day?",
    
    "How many total interactions have been recorded?",
    "How many total interactions were recorded in the year 2010?",
    "What is the total count of observations?",
    "How many interactions occurred in the past year?",
    "How many interactions occurred in the past five years?",
    "How many interactions occurred in the past three years?",
    "What is the average number of interactions observed per year?",
    "What is the total count of transactions?",
    "What is the count of transactions?",
    "What is the total transaction volume observed over the years?",
    "How many transactions occurred in the previous month?",
    "How many transactions took place on the 15th of the previous month?",
    "How many orders were placed on February 20, 2012?",
    "How many transactions were observed between March 2, 2011, and March 31, 2012?",
    "List all order details from the past week.",
    
    "What is the current total customer volume?",
    "How many customers made transactions in the past three months?",
    "What is the size of our customer base?",
    "How many customers are from city XYZ?",
    "How many customers are there in each city?",
    "How many customers are located in London?",
    "What is the average purchase value for customer X?",
    "List the top 5 customers with the highest total purchase value in the past week.",
    "Which customer bought the maximum number of items yesterday from Eastbourne?",
    "What is the average purchase value on Fridays?",
    "Which year had the highest sales?",
    "Which customer had the highest transaction count and gave the highest rating?",
    "In how many cities do we have a presence?",
    "Which city had the highest sales since May last year?",
    "What is the maximum number of items bought by a customer on a single day?",
    
    "How many products are there in each category?",
    "How many categories are there?",
    "Which category has the highest sale?",
    "Which is the highest selling product in each category?",
    "Which product is observed in maximum number of transactions?",
    "Which is the most popular product?",
    "Which category has the highest coverage?",
    
]

pandas_functions = [
    "def orders_in_city_a(df):\n    return df[df['customer_city'] == 'City A'].shape[0]",
    "def avg_price_electronics(df):\n    return df[df['product_category'] == 'Electronics']['price'].mean()",
    "def orders_review_score_5(df):\n    return df[df['review_score'] == 5]",
    "def total_quantity_toys(df):\n    return df[df['product_category'] == 'Toys']['quantity'].sum()",
    "def orders_by_user_id_3(df):\n    return df[df['user_id'] == 3]",
    "def orders_in_city_a(df):\n    return df[df['customer_city'] == 'City A'].shape[0]",
    "def avg_price_electronics(df):\n    return df[df['product_category'] == 'Electronics']['price'].mean()",
    "def orders_review_score_5(df):\n    return df[df['review_score'] == 5]",
    "def total_quantity_toys(df):\n    return df[df['product_category'] == 'Toys']['quantity'].sum()",
    "def orders_by_user_id_3(df):\n    return df[df['user_id'] == 3]",
    "def total_price_orders_user_id_2(df):\n    return df[df['user_id'] == 2]['price'].sum()",
    "def unique_product_categories(df):\n    return df['product_category'].nunique()",
    "def top_3_orders_highest_review_scores(df):\n    return df.nlargest(3, 'review_score')",
    "def min_price_products(df):\n    return df['price'].min()",
    "def orders_placed_on_2024_02_13(df):\n    return df[df['timestamp'] == '2024-02-13'].shape[0]",
    "def orders_in_city_a(df):\n    return df[df['customer_city'] == 'City A'].shape[0]",
    "def avg_price_electronics(df):\n    return df[df['product_category'] == 'Electronics']['price'].mean()",
    "def orders_review_score_5(df):\n    return df[df['review_score'] == 5]",
    "def total_quantity_toys(df):\n    return df[df['product_category'] == 'Toys']['quantity'].sum()",
    "def orders_by_user_id_3(df):\n    return df[df['user_id'] == 3]",
    "def total_price_orders_user_id_2(df):\n    return df[df['user_id'] == 2]['price'].sum()",
    "def unique_product_categories(df):\n    return df['product_category'].nunique()",
    "def top_3_orders_highest_review_scores(df):\n    return df.nlargest(3, 'review_score')",
    "def min_price_products(df):\n    return df['price'].min()",
    "def orders_placed_on_2024_02_13(df):\n    return df[df['timestamp'] == '2024-02-13'].shape[0]",
    "def max_quantity_single_order(df):\n    return df['quantity'].max()",
    "def orders_quantity_greater_than_2(df):\n    return df[df['quantity'] > 2]",
    "def orders_in_february_2024(df):\n    return df[df['timestamp'].str.startswith('2024-02')].shape[0]",
    "def avg_review_score_more_than_1_product(df):\n    return df[df.groupby('order_id')['product_id'].transform('count') > 1]['review_score'].mean()",
    "def orders_user_id_4_city_a(df):\n    return df[(df['user_id'] == 4) & (df['customer_city'] == 'City A')]",
    "def orders_on_2024_02_12_review_score_4_or_higher(df):\n    return df[(df['timestamp'] == '2024-02-12') & (df['review_score'] >= 4)]",
    "def total_price_orders_review_score_5(df):\n    return df[df['review_score'] == 5]['price'].sum()",
    "def orders_price_50_or_higher_review_score_4_or_higher(df):\n    return df[(df['price'] > 50) & (df['review_score'] >= 4)]",
    "def users_more_than_one_order(df):\n    return df['user_id'].value_counts()[df['user_id'].value_counts() > 1].count()",
    "def avg_price_products_each_user(df):\n    return df.groupby('user_id')['price'].mean()",
    "def orders_user_id_1_before_2024_02_11(df):\n    return df[(df['user_id'] == 1) & (df['timestamp'] < '2024-02-11')]",
    "def most_expensive_product_sold(df):\n    return df[df['price'] == df['price'].max()]",
    "def orders_highest_and_lowest_prices(df):\n    return df[(df['price'] == df['price'].max()) | (df['price'] == df['price'].min())]",
    "def total_quantity_products_sold_city_b(df):\n    return df[df['customer_city'] == 'City B']['quantity'].sum()",
    "def orders_users_city_c_review_score_5(df):\n    return df[(df['customer_city'] == 'City C') & (df['review_score'] == 5)]",
    "def orders_between_2024_02_10_and_2024_02_12(df):\n    return df[(df['timestamp'] >= '2024-02-10') & (df['timestamp'] <= '2024-02-12')]",
    "def avg_price_products_each_category(df):\n    return df.groupby('product_category')['price'].mean()",
    "def orders_made_on_weekends(df):\n    return df[pd.to_datetime(df['timestamp']).dt.dayofweek >= 5]",
    "def orders_quantity_less_than_avg_quantity(df):\n    return df[df['quantity'] < df['quantity'].mean()]",
    "def orders_per_user(df):\n    return df['user_id'].value_counts()",
    "def total_revenue_generated(df):\n    return (df['quantity'] * df['price']).sum()",
    "def orders_price_higher_than_avg_price(df):\n    return df[df['price'] > df['price'].mean()]",
    "def total_quantity_products_sold_each_category(df):\n    return df.groupby('product_category')['quantity'].sum()",
    "def orders_in_city_a(df):\n    return df[df['customer_city'] == 'City A'].shape[0]",
    "def avg_price_electronics(df):\n    return df[df['product_category'] == 'Electronics']['price'].mean()",
    
    "def total_interactions(df):\n    return df.shape[0]",
    "def total_interactions_2010(df):\n    return df[df['timestamp'].dt.year == 2010].shape[0]",
    "def total_observations(df):\n    return df.shape[0]",
    "def interactions_last_year(df):\n    return df[df['timestamp'] >= df['timestamp'].max() - pd.DateOffset(years=1)].shape[0]",
    "def interactions_last_5_years(df):\n    return df[df['timestamp'] >= df['timestamp'].max() - pd.DateOffset(years=5)].shape[0]",
    "def interactions_last_three_years(df):\n    return df[df['timestamp'] >= df['timestamp'].max() - pd.DateOffset(years=3)].shape[0]",
    "def average_interactions_per_year(df):\n    return df.shape[0] / df['timestamp'].dt.year.nunique()",
    "def total_transaction_count(df):\n    return df['order_id'].nunique()",
    "def transaction_count(df):\n    return df['order_id'].nunique()",
    "def transaction_volume_over_years(df):\n    return df.groupby(df['timestamp'].dt.year)['price'].sum()",
    "def transactions_last_month(df):\n    return df[df['timestamp'].dt.month == df['timestamp'].max().month].shape[0]",
    "def transactions_on_15th_last_month(df):\n    return df[(df['timestamp'].dt.month == df['timestamp'].max().month - 1) & (df['timestamp'].dt.day == 15)].shape[0]",
    "def orders_on_20th_feb_2012(df):\n    return df[(df['timestamp'].dt.day == 20) & (df['timestamp'].dt.month == 2) & (df['timestamp'].dt.year == 2012)].shape[0]",
    "def transactions_between_2nd_mar_2011_and_31st_mar_2012(df):\n    return df[(df['timestamp'] >= '2011-03-02') & (df['timestamp'] <= '2012-03-31')].shape[0]",
    "def order_details_last_week(df):\n    return df[df['timestamp'] >= df['timestamp'].max() - pd.DateOffset(weeks=1)]",
    
    "def current_customer_volume(df):\n    return df['user_id'].nunique()",
    "def customers_transactions_last_3_months(df):\n    return df[df['timestamp'] >= df['timestamp'].max() - pd.DateOffset(months=3)]['user_id'].nunique()",
    "def customer_base(df):\n    return df['user_id'].nunique()",
    "def customers_in_city_XYZ(df):\n    return df[df['customer_city'] == 'XYZ']['user_id'].nunique()",
    "def customers_in_each_city(df):\n    return df.groupby('customer_city')['user_id'].nunique()",
    "def customers_in_London(df):\n    return df[df['customer_city'] == 'London']['user_id'].nunique()",
    "def average_sale_for_customer_X(df):\n    return df.groupby('user_id')['price'].mean()",
    "def top_5_customers_highest_basket_value_last_week(df):\n    return df[df['timestamp'] >= df['timestamp'].max() - pd.DateOffset(weeks=1)].groupby('user_id')['price'].sum().nlargest(5)",
    "def customer_with_maximum_items_bought_yesterday_from_Eastbourne(df):\n    return df[(df['timestamp'] == df['timestamp'].max() - pd.DateOffset(days=1)) & (df['customer_city'] == 'Eastbourne')].groupby('user_id')['quantity'].sum().idxmax()",
    "def average_basket_value_on_Friday(df):\n    return df[df['timestamp'].dt.dayofweek == 4].groupby('order_id')['price'].sum().mean()",
    "def year_with_highest_sales(df):\n    return df.groupby(df['timestamp'].dt.year)['price'].sum().idxmax()",
    "def customer_with_highest_transaction_and_rating(df):\n    return df.groupby('user_id').agg({'order_id': 'count', 'review_score': 'mean'}).nlargest(1, ['order_id', 'review_score'])",
    "def cities_present(df):\n    return df['customer_city'].nunique()",
    "def city_with_highest_sale_since_May_last_year(df):\n    return df[(df['timestamp'] >= 'LY-05-01') & (df['customer_city'] != '')].groupby('customer_city')['price'].sum().idxmax()",
    "def max_items_bought_by_customer_on_day(df):\n    return df.groupby(['user_id', df['timestamp'].dt.date])['quantity'].sum().max()",
    
    "def total_interactions_recorded(df):\n    return df.shape[0]",
    "def total_interactions_recorded_in_2010(df):\n    return df[df['timestamp'].dt.year == 2010].shape[0]",
    "def total_count_of_observations(df):\n    return df.shape[0]",
    "def interactions_occurred_in_past_year(df):\n    return df[df['timestamp'] >= df['timestamp'].max() - pd.DateOffset(years=1)].shape[0]",
    "def interactions_occurred_in_past_five_years(df):\n    return df[df['timestamp'] >= df['timestamp'].max() - pd.DateOffset(years=5)].shape[0]",
    "def interactions_occurred_in_past_three_years(df):\n    return df[df['timestamp'] >= df['timestamp'].max() - pd.DateOffset(years=3)].shape[0]",
    "def average_interactions_observed_per_year(df):\n    return df.shape[0] / df['timestamp'].dt.year.nunique()",
    "def total_count_of_transactions(df):\n    return df['order_id'].nunique()",
    "def count_of_transactions(df):\n    return df['order_id'].nunique()",
    "def total_transaction_volume_observed_over_years(df):\n    return df.groupby(df['timestamp'].dt.year)['price'].sum()",
    "def transactions_occurred_in_previous_month(df):\n    return df[df['timestamp'].dt.month == df['timestamp'].max().month].shape[0]",
    "def transactions_occurred_on_15th_of_previous_month(df):\n    return df[(df['timestamp'].dt.month == df['timestamp'].max().month - 1) & (df['timestamp'].dt.day == 15)].shape[0]",
    "def orders_placed_on_february_20_2012(df):\n    return df[(df['timestamp'].dt.day == 20) & (df['timestamp'].dt.month == 2) & (df['timestamp'].dt.year == 2012)].shape[0]",
    "def transactions_observed_between_march_2_2011_and_march_31_2012(df):\n    return df[(df['timestamp'] >= '2011-03-02') & (df['timestamp'] <= '2012-03-31')].shape[0]",
    "def list_all_order_details_from_past_week(df):\n    return df[df['timestamp'] >= df['timestamp'].max() - pd.DateOffset(weeks=1)]",
    
    "def current_total_customer_volume(df):\n    return df['user_id'].nunique()",
    "def customers_transactions_past_three_months(df):\n    return df[df['timestamp'] >= df['timestamp'].max() - pd.DateOffset(months=3)]['user_id'].nunique()",
    "def size_of_customer_base(df):\n    return df['user_id'].nunique()",
    "def customers_from_city_XYZ(df):\n    return df[df['customer_city'] == 'XYZ']['user_id'].nunique()",
    "def customers_in_each_city(df):\n    return df.groupby('customer_city')['user_id'].nunique()",
    "def customers_in_London(df):\n    return df[df['customer_city'] == 'London']['user_id'].nunique()",
    "def average_purchase_value_for_customer_X(df):\n    return df.groupby('user_id')['price'].mean()",
    "def top_5_customers_highest_total_purchase_value_past_week(df):\n    return df[df['timestamp'] >= df['timestamp'].max() - pd.DateOffset(weeks=1)].groupby('user_id')['price'].sum().nlargest(5)",
    "def customer_with_maximum_items_bought_yesterday_from_Eastbourne(df):\n    return df[(df['timestamp'] == df['timestamp'].max() - pd.DateOffset(days=1)) & (df['customer_city'] == 'Eastbourne')].groupby('user_id')['quantity'].sum().idxmax()",
    "def average_purchase_value_on_Fridays(df):\n    return df[df['timestamp'].dt.dayofweek == 4].groupby('order_id')['price'].sum().mean()",
    "def year_with_highest_sales(df):\n    return df.groupby(df['timestamp'].dt.year)['price'].sum().idxmax()",
    "def customer_with_highest_transaction_count_and_rating(df):\n    return df.groupby('user_id').agg({'order_id': 'count', 'review_score': 'mean'}).nlargest(1, ['order_id', 'review_score'])",
    "def cities_with_presence(df):\n    return df['customer_city'].nunique()",
    "def city_with_highest_sales_since_May_last_year(df):\n    return df[(df['timestamp'] >= 'LY-05-01') & (df['customer_city'] != '')].groupby('customer_city')['price'].sum().idxmax()",
    "def max_items_bought_by_customer_on_single_day(df):\n    return df.groupby(['user_id', df['timestamp'].dt.date])['quantity'].sum().max()",
    
    "def products_in_each_category(df):\n    return df.groupby('product_category')['product_id'].nunique()",
    "def number_of_categories(df):\n    return df['product_category'].nunique()",
    "def category_with_highest_sale(df):\n    return df.groupby('product_category')['price'].sum().idxmax()",
    "def highest_selling_product_in_each_category(df):\n    return df.groupby('product_category')['product_id'].agg(lambda x: x.value_counts().idxmax())",
    "def product_observed_in_maximum_transactions(df):\n    return df['product_id'].value_counts().idxmax()",
    "def most_popular_product(df):\n    return df['product_id'].value_counts().idxmax()",
    "def category_with_highest_coverage(df):\n    return df.groupby('product_category')['user_id'].nunique().idxmax()",

]

python_problems_df = pd.DataFrame({'question': user_questions, 'solution': pandas_functions}).drop_duplicates()

# msk = np.random.rand(len(python_problems_df)) < 0.8

# train_df = python_problems_df[msk]
# val_df = python_problems_df[~msk]
python_problems_df = python_problems_df.sample(frac=1).reset_index(drop=True)

train_size = int(0.8 * len(python_problems_df))
val_size = int(0.1 * len(python_problems_df))

train_df = python_problems_df[:train_size]
val_df = python_problems_df[train_size:train_size + val_size]
test_df = python_problems_df[train_size + val_size:]

In [4]:
def augment_dataframe(train_df):
    augmented_rows = []
    
    solution_replacements = [
        (("df"), ("pdf")),
        (("df"), ("data")),
        (("df"), ("tbl")),
    ]
    
    question_replacements = [
        (("What is ",), ("",)),
        (("How many ",), ("",)),
        (("List ",), ("",)),
        (("customer",), ("user",)),
        (("user",), ("customer",)),
        (("interactions",), ("transactions",)),
        (("transactions",), ("interactions",)),
        (("average",), ("avg.",)),
        (("years",), ("yr.",)),
    ]
    
    for index, row in train_df.iterrows():
        changes = []
        solution = row["solution"]
        question = row["question"]

        for find_words, replace_words in solution_replacements:
            for find_word, replace_word in zip(find_words, replace_words):
                if find_word in solution:
                    solution = solution.replace(find_word, replace_word)
                    changes.append((find_word, replace_word))

        for find_words, replace_words in question_replacements:
            for find_word, replace_word in zip(find_words, replace_words):
                if find_word in question:
                    question = question.replace(find_word, replace_word)
                    changes.append((find_word, replace_word))
                    
        if changes:
            augmented_row = row.copy()
            augmented_row["solution"] = solution
            augmented_row["question"] = question
            augmented_rows.append(augmented_row)
                
    augmented_train_df = pd.concat([train_df, pd.DataFrame(augmented_rows)], ignore_index=True)
    return augmented_train_df

augmented_train_df = augment_dataframe(train_df)

count_before = len(train_df)
count_after = len(augmented_train_df)

print("Count before augmentation:", count_before)
print("Count after augmentation:", count_after)

Count before augmentation: 80
Count after augmentation: 160


In [5]:
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Training is using {num_gpus} GPU(s).")
    for i in range(num_gpus):
        device = torch.cuda.get_device_properties(i)
        print(f"GPU {i}: {device.name}, Compute Capability: {device.major}.{device.minor}")
        print(f"\tTotal Memory: {device.total_memory / (1024**3):.2f} GB")
        print(f"\tCUDA Cores: {device.multi_processor_count}")
else:
    print("Training is using CPU.")

Training is using 1 GPU(s).
GPU 0: NVIDIA A40, Compute Capability: 8.6
	Total Memory: 44.56 GB
	CUDA Cores: 84


In [6]:
!df -h .

Filesystem      Size  Used Avail Use% Mounted on
/dev/rbd17       24G  9.9G   14G  43% /home/jovyan


In [7]:
model_dir = "codet5_large_python_code_gen"

def clear_directory(directory):
    for root, dirs, files in os.walk(directory, topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))
    os.rmdir(directory)

try:
    clear_directory(model_dir)
except:
    print("Model directory doesn't exist!")

In [9]:
max_question_length = 32
max_solution_length = 64
# model_checkpoint = "Salesforce/codet5-large"
model_checkpoint = "./codet5_large_python_code_gen/checkpoint-20"
strategy = "epoch"
lr = 1e-4
train_batch_size = 16
val_batch_size = 8
decay = 0.01
save_limits = 1
train_iter = 10

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_question_length, max_solution_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_question_length + max_solution_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        inputs = item['question']
        targets = item['solution']

        encoding = self.tokenizer(
            inputs,
            text_target=targets,
            max_length=self.max_length,
            truncation=True,
            padding="max_length"
        )

        return encoding

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

train_dataset = CustomDataset(augmented_train_df, tokenizer, max_question_length, max_solution_length)
val_dataset = CustomDataset(val_df, tokenizer, max_question_length, max_solution_length)
test_dataset = CustomDataset(test_df, tokenizer, max_question_length, max_solution_length)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_checkpoint)

training_args = Seq2SeqTrainingArguments(
    output_dir=model_dir,
    evaluation_strategy=strategy,
    logging_strategy=strategy,
    save_strategy=strategy,
    learning_rate=lr,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=val_batch_size,
    weight_decay=decay,
    save_total_limit=save_limits,
    num_train_epochs=train_iter,
    predict_with_generate=True,
    fp16=False,
    load_best_model_at_end=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# trainer.train()

In [10]:
def print_example(index, test_dataset, predictions, tokenizer):

    example = test_dataset[index]
    
    question = tokenizer.decode(example["input_ids"], skip_special_tokens=True)
    solution = tokenizer.decode(example["labels"], skip_special_tokens=True)
    try:
        prediction = tokenizer.decode(predictions.predictions[index], skip_special_tokens=True)
    except:
        prediction = "No solution! :("

    print("Question:")
    print(question)
    print("\nTrue Solution:")
    print(solution)
    print("\nPredicted Solution:")
    print(prediction)

predictions = trainer.predict(test_dataset, max_new_tokens=max_solution_length)

for idx in range(len(test_dataset)):
    print(idx)
    print_example(idx, test_dataset, predictions, tokenizer)

0
Question:
What is the average purchase value on Fridays?

True Solution:
def average_purchase_value_on_Fridays(df):
    return df[df['timestamp'].dt.dayofweek == 4].groupby('order_id')['price'].sum().mean()

Predicted Solution:

1
Question:
How many interactions happened in the last year?

True Solution:
def interactions_last_year(df):
    return df[df['timestamp'] >= df['timestamp'].max() - pd.DateOffset(years=1)].shape[0]

Predicted Solution:

2
Question:
What is the most expensive product sold?

True Solution:
def most_expensive_product_sold(df):
    return df[df['price'] == df['price'].max()]

Predicted Solution:

3
Question:
How many customers are located in London?

True Solution:
def customers_in_London(df):
    return df[df['customer_city'] == 'London']['user_id'].nunique()

Predicted Solution:

4
Question:
How many orders were placed on 2024-02-13?

True Solution:
def orders_placed_on_2024_02_13(df):
    return df[df['timestamp'] == '2024-02-13'].shape[0]

Predicted Solution

In [11]:
def generate_solution(user_text, trainer, tokenizer):
    input_ids = tokenizer.encode(user_text, max_length=max_question_length, truncation=True, return_tensors="pt")
    input_ids = input_ids.to(trainer.args.device)

    output_ids = trainer.model.generate(input_ids, max_length=max_solution_length, num_beams=4, early_stopping=True)

    generated_solution = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    return generated_solution

user_text = 'How many customers are in each city?'
generated_solution = generate_solution(user_text, trainer, tokenizer)
print("Generated Solution:")
print(generated_solution)

Generated Solution:
    return pt


In [ ]:
# https://www.kaggle.com/code/nayanpatra/seq2seq-code-gen